<a href="https://colab.research.google.com/github/sun-gif/Eluvio-challenge/blob/main/Predict_Movie_scene_boundary_base_on_shots_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Since Movie scence is time dependent, in other words it is sequence of shots,scence boundary not only depend on current shot features, it also depend on previous n shots. so in this notebook, I will use LSTM mode to predict the boundary since LSTM is good to handel time sequences data.

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving tt0068646.pkl to tt0068646 (6).pkl
Saving tt0073195.pkl to tt0073195 (6).pkl
Saving tt0078788.pkl to tt0078788 (6).pkl


Computer limit for  large Ram,  only deal with 3 movie as an sample, 2 as trainning data,1 as testing data

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transformT
from numpy import array

load pickle file, it is already processed as 4 beautiful tensor (audio,action,cast,place) , I will contact those as feature tensor, using ground truth as label, using 1000 as sequences windows(is that simliar to average scence shots?)

In [ ]:
import pickle
def load_file(filename):

  with open(filename,'rb') as f:
      shots = pickle.load(f)
      return shots




In [ ]:
def split_sequences(sequences, y,n_steps):
	X, Y = list(), list()
 
	for i in range(len(sequences)-1):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
    
		# gather input and output parts of the pattern
    
		seq_x, seq_y = sequences[i:end_ix,:], y[end_ix-1]
		X.append(seq_x)
		Y.append(seq_y)
	return array(X), array(Y)

In [ ]:
filenames=['tt0068646.pkl','tt0073195.pkl']
def load_group(filenames):
  movies=[]
  for filename in filenames:
    shots=load_file(filename)
    
    a=shots['action']
    c=shots['cast']
    au=shots['audio']
    p=shots['place']
    t=shots['scene_transition_boundary_ground_truth']
    data=torch.cat((a, c, au,p), 1).numpy()
    y=t.numpy()
    X, Y= split_sequences(data,y, 1000)
    X=torch.from_numpy(X)
    Y=torch.from_numpy(Y)
    movies.append((X,Y))
  return movies

In [ ]:
movies=load_group(filenames)



create an customized Multivariate LSTM Models

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

         
        out = self.fc(out[:, -1, :]) 
        
        return out

Train the model with above two movie, using CrossEntropyLoss() and Adam as optimizer

In [ ]:
num_epochs = 10
learning_rate = 0.1
seq_length=1000
input_dim = 3584
hidden_dim = 2
layer_dim = 1

output_dim = 2

lstm = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
  running_loss=0
  
  for i in range(2):
   X,Y=movies[i]
   
   outputs = lstm(X.float())
   optimizer.zero_grad()
    
    # obtain the loss function
   loss = criterion(outputs, Y.long())
    
   loss.backward()
    
   optimizer.step()
    #if epoch % 100 == 0:
   print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
   
   
          

Epoch: 0, loss: 1.47274
Epoch: 0, loss: 1.35946
Epoch: 1, loss: 1.09982
Epoch: 1, loss: 1.08761
Epoch: 2, loss: 0.89807
Epoch: 2, loss: 0.85330
Epoch: 3, loss: 0.73539
Epoch: 3, loss: 0.66048
Epoch: 4, loss: 0.61338
Epoch: 4, loss: 0.50941
Epoch: 5, loss: 0.52953
Epoch: 5, loss: 0.39644
Epoch: 6, loss: 0.47786
Epoch: 6, loss: 0.31519
Epoch: 7, loss: 0.45068
Epoch: 7, loss: 0.25843
Epoch: 8, loss: 0.44046
Epoch: 8, loss: 0.21951
Epoch: 9, loss: 0.44095
Epoch: 9, loss: 0.19309


In [ ]:
testfiles=['tt0078788.pkl']
testmovies=load_group(testfiles)

Test on the third movie, the actuary is 97%, seems not bad

In [ ]:


for i in range(1):
   X,Y=testmovies[i]
   
   outputs = lstm(X.float())
   optimizer.zero_grad()
    
    # obtain the loss function
   loss = criterion(outputs, Y.long())
    
   loss.backward()
    
   optimizer.step()
    #if epoch % 100 == 0:
   print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
   correct = 0
   total = 0
   lstm.eval()
   y_pred = lstm(X.float())
   test_predict=torch.argmax(y_pred, dim=-1)
   total += Y.size(0)
   correct += (test_predict== Y).sum()

   accuracy = 100 * correct // total
   print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))


Epoch: 9, loss: 0.17662
Iteration: <built-in function iter>. Loss: 0.17661793529987335. Accuracy: 97


# Final Thoughts: Each director has different shoting style , action movie or comdey has differnt tempo. so the scence boundary may also denpend on those feature. so we may can optimzied the result in globling way combine some recomandation systems technique